### Integrales

In [ ]:
"""
    integralsi(a,b,f,n)

Calcula la aproximación de la integral definida de `a` hasta `b` de la función `f` con `n` puntos usando el método de `Simpson compuesto`

"""
function integralsi(a,b,f::Function,n=100)
    @assert (n%2==0) "n debe ser par"  
    h = (b-a)/n
    imp,pa=0,0
    fp=f(a)+f(b)
        
    for i in 1:(n/2)
        pa+=f(a+(2*i-1)*h)
    end
    pa*=4
    for i in 1:(n/2)-1
        imp+=f(a+(2*i)*h)
    end
    imp*=2
    fp+=imp+pa
    
    return (h/3)*fp
end

In [ ]:
"""
    integralsi(x::Vector,f::Vector)

En caso de pasar un número discreto de valores x y f(x).
"""
function integralsi(x::Vector,f::Vector)
    n=length(f)
    I=0
    for i in 1:3:n-n%3
        h= (x[i+1]+x[i+2]-2*x[i])/2
        #t1= (f[i]/((x[i]-x[i+1])*(x[i]-x[i+2])))*((-1/6)*x[i+2]^3 + (1/2)*x[i+2]*x[i]^2 -(1/2)*x[i+1]*x[i+2]^2 + (1/2)*x[i+2]*x[i]^2)
        I+=(h/3)*(f[i]+4*f[i+1]+f[i+2])
    end
    return I
end

In [ ]:
"""
    integraltr(a,b,f::Function,n::Int)

Calcula la aproximación de la integral definida de `a` hasta `b` de la función `f` con `n` puntos usando el método de `Trapecio compuesto`

"""
function integraltr(a,b,f::Function,n=100)
    h=(b-a)/n
    fp=0
    for i in 0:n-1
        fp+=f(a + h*i)
    end
    
    return (h/2)*(f(a)+2*fp+f(b))
    
end 

In [ ]:
"""
    integraltr(x::Vector,f::Vector)

En caso de pasar un número discreto de valores `x` y `f(x)`.
"""
function integraltr(x::Vector,f::Vector)
    n=length(f)
    I=0
    for i in 3:2:n
        #fp=0
        h1 = (x[i-1]-x[i-2])
        h2 = (x[i]-x[i-1])
        f1 = f[i-2]+f[i-1]
        f2 = f[i-1]+f[i]
        I += (h1/2)*(f1) + (h2/2)*(f2)
        #println("h = $h         I[$i] = $I")
    end
    
    return I
end

In [ ]:
"""
    integralpm(a,b,f,n)

Calcula la aproximación de la integral definida de `a` hasta `b` de la función `f` con `n` puntos usando el método de `Punto medio compuesto`

"""
function integralpm(a,b,f::Function,n=100)
    @assert (n%2==0) "n debe ser par"
    h=(b-a)/(n+2)
    fp=0
    for i in 0:n/2
        fp+=f(a + (2*i+1)*h)
    end
    
    return 2*h*fp
end

In [ ]:
"""
    integralpm(x::Vector,f::Vector)

En caso de pasar un número discreto de valores x y f(x).
"""
function integralpm(x::Vector,f::Vector)
    n=length(f)
    I=0
    for i in 2:2:n-1-n%2
        I += f[i+1]*(x[i+2]-x[i])
    end
    return I
end

In [ ]:
"""
    est_errSI(a,b,f,n)

Calcula una cota para el error numérico de la integral definida con el `método de Simpson compuesto` de `a` hasta `b` de la función `f` con `n` puntos 

"""
function est_errSI(a,b,f,n)
     @assert n!=0
    x = a:0.01:b
    f4= der_num3p(x,f.(x))|>j->der_num3p(x,j)|>j->der_num3p(x,j)|>j->der_num3p(x,j) #Cálculo numérico de la f4
    return maximum(abs.(f4.*(  ((b-a)^5)/(18*(n^4))   )  )  )
end

In [ ]:
"""
    est_errTR(a,b,f,n)

Calcula una cota para el error numérico de la integral definida con el `método de trapecio compuesto` de `a` hasta `b` de la función `f` con `n` puntos 

"""
function est_errTR(a,b,f,n)
     @assert n!=0
    x = a:0.01:b
    f4= der_num3p(x,f.(x))|>j->der_num3p(x,j) #Cálculo numérico de la f2
    return maximum(abs.(f4.*(    ((b-a)^3)/(12*(n^2))    )  )  )
end

In [ ]:
"""
    est_errPM(a,b,f,n)

Calcula una cota para el error numérico de la integral definida con el `método de punto medio compuesto` de `a` hasta `b` de la función `f` con `n` puntos 

"""
function est_errPM(a,b,f,n)
    @assert n!=0
    x = a:0.01:b
    f4= der_num3p(x,f.(x))|>j->der_num3p(x,j) #Cálculo numérico de la f2
    return maximum(abs.(f4.*(  ((b-a)^3)/(6*(n^2))  )  )  )
end

In [ ]:
"""
    estim_n(a,b,f,tol,met)

Devuelve la cantidad de puntos para calcular la integral indefinida de `a` hasta `b` de la función `f` con un error menor a `tol`
# Argumentos
- `met`: "SI" para el método de `Simpson`
- `met`: "TR" para el método de `Trapecio`
- `met`: "PM" para el método de `Punto Medio`

"""
function estim_n(a,b,f,tol,met="SI")
    E,i=tol+1,2
    if met=="SI"
        while E > tol
            i+=1
            E=est_errSI(a,b,f,i)
        end
    elseif met=="PM"
        while E > tol
            i+=1
            E=est_errPM(a,b,f,i)
        end
    elseif met=="TR" 
        while E > tol
            i+=1
            E=est_errTR(a,b,f,i)
        end
    end        
    return i+i%2
end

### Ecuaciones Diferenciales:

In [ ]:
function paso_euler(f,y0,t0,h,p=nothing)
    return (y0 + h*f(y0,t0,p))
end
"""
    dif_euler(f,a,b,y0,h,p)
Calcula la ecuación diferencial dy/dt = `f`(y(t),`t`,`p`) en el intervalo [`a`,`b`], con condiciones iniciales y(`a`)=`y0`, en intervalos de ancho `h`
mediante el método de Euler
"""
function dif_euler(f,a,b,y0,h,p=nothing)
    N = round(Int,(b-a)/h)
    
    vec_w,vec_t=Vector{Float64}(undef,N+1),Vector{Float64}(undef,N+1)
    
    vec_w[1]=y0
    vec_t[1]=a
    
    for i in 2:1:N+1
        vec_t[i]=a+i*h
        vec_w[i]=paso_euler(f,vec_w[i-1],vec_t[i-1],h,p)
    end
    
    return vec_w,vec_t
end
        

In [ ]:
function paso_rk2(f,y0,t0,h,p=nothing)
    return y0 + h * f(  y0+(h/2)  *  f(y0,t0,p)  ,  t0+h/2  ,  p)
end
"""
    dif_rk2(f,a,b,y0,h,p)
Calcula la ecuación diferencial dy/dt = `f`(y(t),`t`,`p`) en el intervalo [`a`,`b`], con condiciones iniciales y(`a`)=`y0`, en intervalos de ancho `h`
mediante el método de Runge-Kutta de segundo orden
"""
function dif_rk2(f,a,b,y0,h,p=nothing)
    N = round(Int,(b-a)/h)
    
    vec_w,vec_t =Vector{Float64}(undef,N+1),Vector{Float64}(undef,N+1)
    vec_w[1],vec_t[1]=y0,a
    
    for i in 2:1:N+1
        vec_t[i]=a+i*h
        vec_w[i]= paso_rk2(f,vec_w[i-1],vec_t[i-1],h,p) 
    end
    
    return vec_w,vec_t
    
end

In [ ]:
function paso_rk4(f,y0,t0,h,p=nothing)
    k1=h*f(y0  ,  t0  ,p)
    k2=h*f(y0+k1/2  ,t0+h/2  ,p)
    k3=h*f(y0+k2/2  ,t0+h/2  ,p)
    k4=h*f(y0+k3  ,t0+h  ,p)
    
    return y0 + (k1+  2*k2  +  2*k3  +  k4)/6

end
"""
    dif_rk4(f,a,b,y0,h,p)
Calcula la ecuación diferencial dy/dt = `f`(y(t),`t`,`p`) en el intervalo [`a`,`b`], con condiciones iniciales y(`a`)=`y0`, en intervalos de ancho `h`
mediante el método de Runge-Kutta de segundo orden
"""
function dif_rk4(f,a,b,y0,h,p=nothing)
    N = round(Int,(b-a)/h)

    vec_w=Vector{Float64}(undef,N+1)
    vec_t=Vector{Float64}(undef,N+1)

    vec_w[1],vec_t[1]=y0,a

    for i in 2:1:N+1
        vec_t[i]=a+i*h
        vec_w[i] = paso_rk4(f,vec_w[i-1],vec_t[i-1],h,p)
    end

    return vec_w,vec_t

end

In [ ]:
#=
function paso_euler(f,y0,t0,h,p=nothing)
    return (y0 + h*f(y0,t0,p))
end
=#


function dif_euler_multi(f,a,b,y0,h,p=nothing)
    n = round(Int,(b-a)/h)
    
    vec_w=zeros(length(y0),n)
    vec_t=zeros(n)
    
    vec_w[:,1]=y0
    vec_t[1]=a
    
    for i in 2:1:n
        vec_t[i]=vec_t[i-1]+h
        vec_w[:,i]=paso_euler(f,vec_w[:,i-1],vec_t[i-1],h,p)
    end
    
    return vec_w,vec_t
    
    
end

In [ ]:
#=function paso_rk4(f,y0,t0,h,p=nothing)
    k1=h*f(y0  ,  t0  ,p)
    k2=h*f(y0+k1/2  ,t0+h/2  ,p)
    k3=h*f(y0+k2/2  ,t0+h/2  ,p)
    k4=h*f(y0+k3  ,t0+h  ,p)
    
    return y0 + (k1+  2*k2  +  2*k3  +  k4)/6

end
=#
"""
    dif_rk4_multi(f,a,b,y0,h,p=nothing)
Calcula la/s ecuaciones diferenciales de f(y1,y2,...yn,t,p) con condiciones iniciales y0 = [y1o,y2o,...yno]
con paso de integración h con el método de Runge-Kutta 4.
 Si m= (a+b)/h
devuelve: 
vec_w: [Y11 Y21 ... Yn1;
        Y12 Y22 ... Yn2;
         .   .  ...  . ;
        Y1m Y2m ... Ynm]

vec_t: [a+h, a+2h, ... a+mh]
"""
function dif_rk4_multi(f,a,b,y0,h,p=nothing)
    N = round(Int,(b-a)/h)

    vec_w=zeros(N,length(y0))
    vec_t=zeros(N)
    
    vec_w[1,:]=y0
    vec_t[1]=a

    for i in 2:1:N
        vec_t[i]=a+i*h
        vec_w[i,:]= paso_rk4(f,vec_w[i-1,:],vec_t[i-1],h,p)
    end

    return vec_w,vec_t

end 